<a href="https://colab.research.google.com/github/gahongayirebelise/NLP-FELLOWSHIP/blob/main/webscrapping_igihe_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import numpy as np
import re
from bs4 import BeautifulSoup as bs
from google.colab import drive
import pandas as pd

create the sequencial (daily) links from january to december 2022

In [42]:
unsortedLinks = []

for month in range(1, 11):
  for day in range(1, 32):
    l = "http://archive.org/wayback/available?url=igihe.com&timestamp=2022{:02d}{:02d}".format(month, day)
    response = requests.get(l).json()
    if  response['archived_snapshots']:
      unsortedLinks.append(response['archived_snapshots']['closest']["url"])

check if they are some duplicated links and get rid of them

In [43]:
len(unsortedLinks)

303

In [44]:
arr = np.array(unsortedLinks)
series = pd.Series(arr)
unsortedLinks = series.unique()

In [45]:
serious_links = {}

In [46]:
# link = "https://web.archive.org/web/20220615035616/https://www.igihe.com/amakuru/"
def process_links(link):
  page = requests.get(link)
  soup = bs(page.text, "html.parser")
  
  b = soup.body
  get_titles = [link+re.sub("amakuru/","", a['href']) for span in b.find_all("span", class_ = "homenews-title") for a in span.find_all("a")]
  
  # get only links whose date is in current year
  pattern = re.compile(r"\d{8,}?")
  found_date = pattern.search(link).group()
  if found_date[3] != str(2):
    pass
  else:
    file_title = f"igihe-{found_date[0:4]}-{found_date[4:6]}-{found_date[6:8]}"
    try:
      for title in get_titles:
        serious_links[file_title].append(title)
    except KeyError:
      serious_links[file_title] = []
      for title in get_titles:
        serious_links[file_title].append(title)

In [47]:
for link in unsortedLinks:
  process_links(link)

In [48]:
serious_links

{'igihe-2022-01-13': ['http://web.archive.org/web/20220113045331/https://igihe.com/u-rwanda/article/kigali-imodoka-yarenze-umuhanda-igwa-mu-mugezi',
  'http://web.archive.org/web/20220113045331/https://igihe.com/u-rwanda/article/miliyoni-38-zigiye-gushorwa-mu-bikorwa-biteza-imbere-abaturiye-pariki-z-igihugu',
  'http://web.archive.org/web/20220113045331/https://igihe.com/u-rwanda/article/uburyo-kayibanda-yigishijwe-kwigomeka-n-uruhare-rwa-kinyamateka-mu',
  'http://web.archive.org/web/20220113045331/https://igihe.com/umuco/article/inkomoko-y-insigamugani-yigize-idebe',
  'http://web.archive.org/web/20220113045331/https://igihe.com/u-rwanda/article/huye-ibyo-wamenya-kuri-gahunda-yo-gukingira-covid-19-ku-mashuri',
  'http://web.archive.org/web/20220113045331/https://igihe.com/imikino/football/article/patrice-evra-yahishuye-ko-hari-abatinganyi-bakinanye-ndetse-muri-buri-kipe',
  'http://web.archive.org/web/20220113045331/https://igihe.com/u-rwanda/article/musanze-imbogo-ebyiri-zarwanye-ku

In [49]:
def writeToFile(paragraphs, file):
  with open(file, "a+") as f:
    for line in paragraphs:
      f.write(line)

In [50]:
def get_page():
  count = 1
  for key in list(serious_links.keys()):
    for link in set(serious_links[key]):
      p = requests.get(link).content
      soup = bs(p, "html.parser")
      spans = soup.find_all("div", class_= "fulltext margintop10")
      paragraphs = [p.get_text() for span in spans for p in span.find_all("p")]
      writeToFile(paragraphs = paragraphs, file = f"/content/drive/MyDrive/NLP fellowship/week1 tasks/{key}.txt")
    print(f"file {count} created")
    count += 1